In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta.volatility import average_true_range
from ta.trend import SMAIndicator

In [2]:
# Load and preprocess data
def preprocess_data(file_path):
    data = pd.read_csv(file_path, parse_dates=True, index_col='Date')
    data = data.dropna()
    return data

In [3]:
# Calculate SuperTrend
def supertrend(data, atr_period, atr_multiplier):
    high = data['High']
    low = data['Low']
    close = data['Close']
    
    # Calculate ATR
    data['atr'] = average_true_range(high, low, close, window=atr_period)
    
    # Calculate upper and lower bands
    data['upperband'] = ((high + low) / 2) + (atr_multiplier * data['atr'])
    data['lowerband'] = ((high + low) / 2) - (atr_multiplier * data['atr'])
    
    # Initialize SuperTrend
    data['supertrend'] = 0.0
    data['supertrend'][0] = data['upperband'][0]
    
    # Calculate SuperTrend
    for i in range(1, len(data)):
        if close[i] > data['upperband'][i-1]:
            data['supertrend'][i] = data['upperband'][i]
        elif close[i] < data['lowerband'][i-1]:
            data['supertrend'][i] = data['lowerband'][i]
        else:
            data['supertrend'][i] = data['supertrend'][i-1]
    
    return data

In [4]:
# Define the trading strategy
def trading_strategy(data, initial_cash=10000):
    cash = initial_cash
    position = 0  # 0 means no position, 1 means long, -1 means short
    data['cash'] = cash
    data['position'] = position
    data['portfolio_value'] = cash
    highest_value = cash
    lowest_value = cash
    
    for i in range(1, len(data)):
        if position == 0:
            # Check for long entry
            if data['supertrend'][i] > data['Close'][i]:
                position = 1
                entry_price = data['Close'][i]
                print(f"Entering long at {entry_price} on {data.index[i]}")
            # Check for short entry
            elif data['supertrend'][i] < data['Close'][i]:
                position = -1
                entry_price = data['Close'][i]
                print(f"Entering short at {entry_price} on {data.index[i]}")
        elif position == 1:
            # Check for long exit
            if data['supertrend'][i] < data['Close'][i]:
                position = 0
                cash += (data['Close'][i] - entry_price) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting long at {data['Close'][i]} on {data.index[i]}")
        elif position == -1:
            # Check for short exit
            if data['supertrend'][i] > data['Close'][i]:
                position = 0
                cash += (entry_price - data['Close'][i]) * 10  # Assuming we buy/sell 10 units
                highest_value = max(highest_value, cash)
                lowest_value = min(lowest_value, cash)
                print(f"Exiting short at {data['Close'][i]} on {data.index[i]}")
        
        data.at[data.index[i], 'cash'] = cash
        data.at[data.index[i], 'position'] = position
        data.at[data.index[i], 'portfolio_value'] = cash + position * (data['Close'][i] - entry_price) * 10
    
    return data

In [5]:
# Run the backtest
if __name__ == "__main__":
    file_path = 'your_data.csv'  # Replace with your data file path
    data = preprocess_data(file_path)
    atr_period = 10
    atr_multiplier = 3.0
    data = supertrend(data, atr_period, atr_multiplier)
    results = trading_strategy(data)
    
    # Plot the results
    plt.figure(figsize=(12, 8))
    plt.plot(results['portfolio_value'], label='Portfolio Value')
    plt.title('Portfolio Value Over Time')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value')
    plt.legend()
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'your_data.csv'